In [18]:
import os
import pandas as pd

# Specify the target date
target_date_str = "2018-12-29"
target_date = pd.to_datetime(target_date_str)

# Construct the file name (assuming pattern "post_despacho_YYYYMMDD.parquet")
filename = f"post_despacho_{target_date.strftime('%Y%m%d')}.parquet"

# Define the raw folder path (adjust if necessary)
raw_folder = r"..\data\raw"
file_path = os.path.join(raw_folder, filename)

# Read the parquet file
df = pd.read_parquet(file_path)
print("Original DataFrame preview:")
df.head(3)


Original DataFrame preview:


,GRUPOS,INDICE,GRUPO,EMPRESA,CENTRAL,FECHA,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
0,1 - Térmica,1,Térmica,AES ANDRES,AES ANDRES,2018-12-29T00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1 - Térmica,1,Térmica,AES ANDRES,GENERACIÓN DE EMERGENCIA AES ANDRÉS,2018-12-29T00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1 - Térmica,1,Térmica,BERSAL,BERSAL,2018-12-29T00:00:00,16.0,16.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,0.0,0.0,6.0,6.0,6.0,6.0,16.0,6.0,16.0,6.0,0.0,0.0,0.0


In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
df[df['CENTRAL'] == 'HATILLO'].head()

,GRUPOS,INDICE,GRUPO,EMPRESA,CENTRAL,FECHA,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
61,3 - Hidroeléctrica,3,Hidroeléctrica,EGEHID,HATILLO,2018-12-29T00:00:00,7.26,7.13,7.15,6.83,6.91,7.11,7.13,7.01,7.06,6.97,6.77,6.7,7.16,6.99,7.07,7.16,7.14,6.92,7.12,7.02,22.0,6.77,7.1,7.1


In [122]:
# Define the range of columns to set to zero (from H10 to H15)
cols_to_zero = [f"H{i}" for i in range(1, 25)]

# Update the DataFrame: For rows where CENTRAL is "ESTRELLA DEL MAR 2 SFO", set the specified columns to 0.0
df.loc[df['CENTRAL'] == 'ESTRELLA DEL MAR 2 CGN', cols_to_zero] = 0.0


In [19]:
mask = (df['CENTRAL'] == 'HATILLO')

# Set the value in H20 to zero for the matching ro
df.loc[mask, 'H22'] = 7.68

In [20]:
# Save the modified DataFrame back to the same location in the raw folder
df.to_parquet(file_path, index=False)
print(f"Modified DataFrame saved back to: {file_path}")

Modified DataFrame saved back to: ..\data\raw\post_despacho_20181229.parquet
